# **Esame del 30 Giugno 2023**

- Non si possono consultare libri, note, ed ogni altro materiale o persone durante
l’esame ad eccezione delle funzioni Python fornite.

- Risolvere i seguenti esercizi con l’ausilio di Python.

- La durata del compito è di 90 minuti.

- Questo esame ha 3 domande, per un totale di 30/30 punti.

- Svolgere gli esercizi marcati con [T] su fogli protocollo, indicando: nome, cognome, codice persona
e data

# **Esercizio 1 (punti 10)**

Si consideri la seguente funzione

$$f(x) = x\sin(x)$$

per $x \in [-1,1]$.

**(a) (1 punto) [P]** Si rappresenti $f$ in Python e si identifichi il valore $\alpha$ tale che $f(\alpha)=0$.




In [3]:
import functions # prendiamo tutte le funzioni dalla libreria fornita
import numpy as np
import matplotlib.pyplot as plt



**(b) (3 punti) [T]** Derivare il Metodo di Newton per la ricerca degli zeri di una funzione, riportando anche le sue proprietà di convergenza.

**(c) (2 punti) [P]** Applicare il metodo di Newton per il calcolo di $\alpha$, partendo da una guess iniziale pari a $x^0 = 0.5$ e impostare una tolleranza pari a $tol=1e-8$. Rappresentare in scala semilogaritmica l'errore ottenuto e commentare alla luce della teoria.

**(d) (2 punti) [T]** Si proponga una modifica al metodo di Newton per il calcolo degli zeri di molteplicità algebrica superiore a 1.

**(e) (2 punti) [P]** Si estenda opportunamente la function del metodo di Newton in modo da implementare quanto proposto al punto precedente. Utilizzando questa nuova funzione, ripetere quanto fatto al punto (c), sovrapponendo gli errori sullo stesso grafico. Commentare i risultati ottenuti.

In [29]:
# Creiamo qui la function di newton modificato


In [2]:
# Applichiamo i due metodi separatamente e confrontiamo i risultati


# **Esercizio 2 (10 punti)**

Si consideri la seguente matrice

$$A=\begin{bmatrix} 1&1&1&1&1\\
1 & 2& 3&4&5\\
1&3&6&10&15\\
1&4&10&20&35\\
1&5&15&35&70 \end{bmatrix} $$

che si può ottenere su Python tramite la libreria **scipy.linalg**, con comando **pascal(n)**, dove $n$ è la dimensione della matrice.

**(a) (3 punti) [P+T]** Enunciare la condizione necessaria e sufficiente per l'esistenza e unicità della fattorizzazione $LU$ e verificare (con opportuni comandi) che è soddisfatta per la matrice $A$ di dimensione $n=5$.

In [4]:
import numpy as np
import scipy.linalg as sl
import matplotlib.pyplot as plt
import functions



**(b) (3 punti) [P]** Data la soluzione esatta $x = [1,1,...,1]^T$ costruire il termine noto $b$ e risolvere il sistema lineare $Ax = b$ utilizzando:

i) la fattorizzazione $LU$,

ii) i metodi di sostituzione in avanti e all'indietro disponibili nella libreria **functions.py** fornita.

Verificare se è stato effettuato il pivoting.

In [5]:
import numpy as np


**(c) (4 punti) [P+T]** Ripetere i passaggi al punto precedente per matrici di dimensione $n = 10$, $15$ e $20$. Per ognuno dei casi calcolare la norma dell’errore relativo e il condizionamento della matrice e rappresentarli su due grafici in scala logaritmica. Commentare i risultati alla luce della teoria.

# **Esercizio 3 (10 punti)**

Si consideri il seguente problema già affrontato nella parte 1 dell'esame.

$$\begin{cases}
\partial_t u - \partial_{xx} u = 1 & \text{se } x \in (0,\pi),\,\, t>0\\
u(t,0) = u(t,\pi) = 0 & \text{se } t>0\\
u(0,x) = 0 & \text{se } x\in (0,\pi)
\end{cases}
$$

**(a) (5 punti) [T]** Introdurre brevemente l'approssimazione del problema con il metodo degli elementi finiti in spazio e un generico $\theta-$metodo in tempo. Derivare l'espressione matriciale (senza dettagliare i singoli elementi delle matrici).

**(b) (3 punti) [P]** Si consideri una griglia di ampiezza uniforme $h = \pi/20$. Si risolva il problema con il metodo di Eulero Esplicito, usando un passo temporale $\Delta t=0.25$, quindi si ripeta il calcolo con $\Delta t=0.0025$. Utilizzare la function $\texttt{heatsolve}$ fornita di seguito. Si rappresentino le soluzioni e si commentino alla luce della teoria.

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from fem import install

install()
from fem import Line, generate_mesh, FEspace, assemble, interpolate, deriv, dx, ds, DirichletBC, applyBCs, dof2fun, fun2dof, dofs, plot

def heatSolve(D,f,u0,L,h,T,dt,theta):
  """"
  Input:
     D      (float)                  Coefficiente di diffusione (positivo).
     f      (lambda function)        Forzante. Si assume f = f(x,t).
     u0     (lambda function)        Condizione iniziale.
     L      (float)                  Lunghezza dell'intervallo spaziale.
     h      (float)                  Passo della griglia spaziale.
     T      (float)                  Tempo finale
     dt     (float)                  Passo temporale.
     theta  (float)                  Parametro del theta-metodo.

  Output:
    V                               spazio elementi finiti
    u     (numpy.ndarray)-> matrix  Matrice contenente la soluzione
                                    approssimata del problema. Uij
                                    approssima u(dof_i, tj): ogni colonna è un
                                    tempo fissato.
    t      (numpy.ndarray)-> vector Griglia temporale.
  """""
  # costruisco il dominio
  domain = Line(0, L)
  # costruisco la mesh
  mesh = generate_mesh(domain, stepsize = h)
  # costruisco lo spazio FEM di grado 1
  V = FEspace(mesh, 1)

  # costruisco la griglia temporale
  nt = np.ceil(T/dt)+1
  t = np.zeros(int(nt))

  # initializzo la soluzione
  u = np.zeros((dofs(V).size, int(nt)))

  # definisco la condizione iniziale
  u0h = fun2dof(interpolate(u0,V))
  u[:, 0] = u0h

  # matrice di massa
  def m(u, v):
    return u*v*dx
  # assemblaggio matrice di massa
  M = assemble(m, V)

  # matrice di diffusione
  def a(u,v):
    return deriv(u)*deriv(v)*dx
  # assemblaggio matrice di diffusione
  A = D*assemble(a,V)

  # ciclo temporale
  for n in range(int(nt)-1):
    # costruzioni termini noti al tempo dt e dt+1
    t_old = n*dt
    t_new = (n+1)*dt

    fold = lambda x: f(x,t_old)
    fnew = lambda x: f(x,t_new)

    fold_h = interpolate(fold, V)
    def lold(v):
      return fold_h*v*dx
    Fold = assemble(lold, V)

    fnew_h = interpolate(fnew, V)
    def lnew(v):
      return fnew_h*v*dx
    Fnew = assemble(lnew, V)

    # condizioni al bordo omogenee di tipo dirichlet
    def isLeftNode(x):
      return x < 1e-12

    def isRightNode(x):
      return x > L - 1e-12

    dbc1 = DirichletBC(isLeftNode,  0.0)
    dbc2 = DirichletBC(isRightNode, 0.0)

    A = applyBCs(A, V, dbc1, dbc2)
    M = applyBCs(M, V, dbc1, dbc2)
    Fold = applyBCs(Fold, V, dbc1, dbc2)
    Fnew = applyBCs(Fnew, V, dbc1, dbc2)

    # Costruzione del sistema lineare e sua risoluzione
    B = (M/dt+theta*A)
    b = (M/dt-(1-theta)*A)@u[:,n] + theta*Fnew +(1-theta)*Fold

    from scipy.sparse.linalg import spsolve

    u[:,n+1] = spsolve(B, b)
    t[n+1] = t_new

  return V,u,t

In [8]:
import numpy as np
from fem import xtplot



**(c) (2 punti) [P]** Si risolva ora il problema con il metodo di Eulero Implicito, scegliendo
$\Delta t = 0.25$. Si rappresenti il valore della soluzione numerica al centro del dominio,
in tempo: $u_h (t, \frac{\pi}{2} )$ (da estrarre con un metodo a scelta). Cosa si osserva?